In [93]:
# import libraries
import requests
import json
import pandas as pd
import re
import os
from dotenv import load_dotenv
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime
from pytz import timezone


In [94]:
# Getting API key
load_dotenv("api_key.env")
api_key = os.getenv("API_KEY")

In [95]:
def create_connection(schema):
    host = "127.0.0.1"
    user = "root"
    load_dotenv("sql_password.env")
    password = os.getenv("SQL")
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

connection_string = create_connection("cities")

In [96]:
class WeatherDataUpdater: 
    def __init__(self, connection_string, api_key):
        self.connection_string = connection_string
        self.api_key = api_key

    def retrieve_and_send_data(self):
        cities_df = self.fetch_cities_data()
        weather_df = self.fetch_weather_data(cities_df)
        self.store_weather_data(weather_df)
        return "Data has been updated"

    def fetch_cities_data(self):
        return pd.read_sql("cities", con=self.connection_string)

    def fetch_weather_data(self, cities_df):
        berlin_timezone = timezone('Europe/Berlin')
        # Create an empty dictionary to hold the data
        weather_dict = {"city_id": [],
                        "city": [],
                        "latitude": [],
                        "longitude": [],
                        "temp": [],
                        "feels_like": [],
                        "time_forecasted": [],
                        "weather": [],
                        "weather_description": [],
                        "rain_last_3h": [],
                        "humidity": [],
                        "wind_speed": [],
                        "clouds": [],
                        "data_retrieved_at": []
                        }

        for _, city in cities_df.iterrows():
            city_name = city["city_name"]
            city_id = city["city_id"]
            url = f"https://api.openweathermap.org/data/2.5/forecast?q={city_name}&units=metric&appid={api_key}"
            response = requests.get(url)
            json_data = response.json()
            weather_df = pd.json_normalize(json_data["list"])
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for i in range(0, len(weather_df), 8):
                weather_dict['city_id'].append(city_id)
                weather_dict['city'].append(json_data['city']['name'])
                weather_dict["latitude"].append(json_data['city']['coord']['lat'])
                weather_dict["longitude"].append(json_data['city']['coord']['lon'])
                weather_dict["temp"].append(json_data['list'][i]['main']['temp'])
                weather_dict["feels_like"].append(json_data["list"][i]["main"]["feels_like"])
                weather_dict["time_forecasted"].append(json_data["list"][i]['dt_txt'])
                weather_dict["weather"].append(json_data["list"][i]['weather'][0]['main'])
                weather_dict["weather_description"].append(json_data["list"][i]['weather'][0]['description'])
                weather_dict["rain_last_3h"].append(json_data["list"][i].get("rain", {}).get("3h", 0))
                weather_dict["humidity"].append(json_data["list"][i]["main"]["humidity"])
                weather_dict["wind_speed"].append(json_data["list"][i]["wind"]["speed"])
                weather_dict["clouds"].append(json_data["list"][i]["clouds"]["all"])
                weather_dict["data_retrieved_at"].append(retrieval_time)

        weather_dataframe = pd.DataFrame(weather_dict)
        weather_dataframe["time_forecasted"] = pd.to_datetime(weather_dataframe["time_forecasted"])
        weather_dataframe["data_retrieved_at"] = pd.to_datetime(weather_dataframe["data_retrieved_at"])
        return weather_dataframe
        pass

    def store_weather_data(self, weather_dataframe):
         #print("Storing weather data:")
         #print(weather_df.head())  # Print the first few rows to check the data being passed
         weather_dataframe.to_sql("weather",
                          con=self.connection_string,
                          if_exists="append",
                          index=False)


In [97]:
weather_updater = WeatherDataUpdater(connection_string, api_key)
weather_updater.retrieve_and_send_data()

'Data has been updated'